In [1]:
%run my_import

uri = socket.gethostname()+".local:5555"
#uri = "com-exp.local:5555"
#uri = "ipc3-xeno.local:5555"

#uri = "localhost:5555"

io = set_uri(uri)
io.debug = False
scan_ids = list(reply_cmd(master_cmd_get_slave_descr).keys())
print('Found ESC ids {}'.format(scan_ids))

Python path : /home/amargan/work/anaconda3/envs/repl/bin/python
Python version : 3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:28:47) [GCC 11.3.0]


ModuleNotFoundError: No module named 'header_pb2'

NameError: name 'set_uri' is not defined

In [ ]:
iD = 1
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))

In [ ]:
sdo_filter(["Position_actual_value_x6064","Target_position_x607A"],sdo_dict)

In [ ]:
sdo = ['Statusword_x6041','Position_actual_value_x6064','Target_position_x607A'] #torque','motor_pos','link_pos','Min_pos','Max_pos','Max_cur']
read_sdo(sdo,[iD])[iD]

In [ ]:
#io.doit(ctrl_cmd_start.set_bid(bId).set_value(0x3B).set_gains(u'POSITION',200,0,15,0,0,0))
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0x3B))

In [ ]:
reply = io.doit(ctrl_cmd_stop.set_bid(iD))

In [ ]:
data = read_sdo(sdo,[iD])[iD]
pos_ref = data['Position_actual_value_x6064']
io.doit(ctrl_cmd_set_position.set_bid(iD).set_value(0.01))
#read_sdo(sdo,[iD])[iD]

In [ ]:
# read from pdo publisher
start_pos = 0.01


In [1]:
import asyncio

import zmq
import zmq.asyncio
import pprint
import json
from protobuf_to_dict import protobuf_to_dict

#import sys
#sys.path.append('/home/amargan/work/code/ecat_dev/ecat_master_advr/build/protobuf')
#import ecat_pdo_pb2
from ecat_repl import ecat_pdo_pb2

# Define a global variable to store the last received data
last_recv_data = None,None

def process_data(msg_id, data):
    rx_pdo = ecat_pdo_pb2.Ec_slave_pdo()
    rx_pdo.ParseFromString(data)
    pb_dict = protobuf_to_dict(rx_pdo)
    return msg_id,pb_dict

#from utils import get_address
def get_address():
    protocol = "tcp"
    address = "127.0.0.1"
    port = 9601
    return f"{protocol}://{address}:{port}"

async def zsub():
    global last_recv_data
    subscriber = zmq.asyncio.Context().socket(zmq.SUB)
    subscriber.connect(get_address())
    subscriber.subscribe(b"")

    while True:
        msg_id, data = await subscriber.recv_multipart()
        # store the last received data
        last_recv_data = process_data(msg_id, data)
        #pprint.pprint((msg_id, pb_dict))



ImportError: cannot import name 'ecat_pdo_pb2' from 'ecat_repl' (/home/amargan/work/code/python/ecat_repl/ecat_repl/__init__.py)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Create a background task for the zsub coroutine
zsub_task = asyncio.create_task(zsub())

In [ ]:
last_recv_data

In [ ]:
pprint.pprint(last_recv_data[1]['header'])